In [5]:
import os

# Dataset for the OAI images
data_folder1 = '/playpen-raid/data/OAI/0.C.2'
data_folder2 = '/playpen-raid/data/OAI/0.E.1'

data1 = os.listdir(data_folder1)
data2 = os.listdir(data_folder2)

print('Number of patients in total: ', len(set.union(set(data1), set(data2))))

Number of patients in total:  4796


In [12]:
# Dataset splitting
import pandas as pd

# dataset split for OAI dataset
pth_file = "/playpen-raid/data/OAI/contents.0E1.csv"
pth_df = pd.read_csv(pth_file)

SAG_3D_DESS_RIGHT = "SAG_3D_DESS_RIGHT"
SAG_3D_DESS_LEFT = "SAG_3D_DESS_LEFT"
sag_3d_dess_right_cases = pth_df[pth_df["SeriesDescription"] == SAG_3D_DESS_RIGHT]
sag_3d_dess_left_cases = pth_df[pth_df["SeriesDescription"] == SAG_3D_DESS_LEFT]

# get all the PIDs
PIDs = set(pth_df['ParticipantID'])

# get PIDs of OAI-ZIB dataset
ZIB_PID_to_Folder = {}
with open('/playpen-raid2/qinliu/data/OAI-ZIB/doc/oai_mri_paths.txt') as f:
    lines = f.readlines()
    for line in lines:
        PID, folder = line.split(' ')[:2]
        PID = int(PID[:-1])
        ZIB_PID_to_Folder[PID] = folder

# dataset splitting according to PID
import random
random.seed(0)

ZIB_PIDs = list(ZIB_PID_to_Folder.keys())
random.shuffle(ZIB_PIDs)

ZIB_PIDs_train = ZIB_PIDs[:407]
ZIB_PIDs_val = ZIB_PIDs[407:457]
ZIB_PIDs_test = ZIB_PIDs[457:507]
print('Split for ZIB: ', len(ZIB_PIDs_train), len(ZIB_PIDs_val), len(ZIB_PIDs_test))

rest_PIDs = list(set(PIDs) - set(ZIB_PIDs))
random.shuffle(rest_PIDs)
rest_PIDs_train = rest_PIDs[:3431]
rest_PIDs_val = rest_PIDs[3431:3860]
rest_PIDs_test = rest_PIDs[3860:]

train_set = set(ZIB_PIDs_train + rest_PIDs_train)
val_set = set(ZIB_PIDs_val + rest_PIDs_val)
test_set = set(ZIB_PIDs_test + rest_PIDs_test)
print('Split for OAI: ', len(train_set), len(val_set), len(test_set))

# save the split to csv files
pd_data = []
for _, row in sag_3d_dess_right_cases.iterrows():
    folder, PID, right_or_left = row['Folder'], row['ParticipantID'], 'RIGHT' if row['SeriesDescription'] == 'SAG_3D_DESS_RIGHT' else 'LEFT'
    split = '?'
    if PID in train_set:
        split = 'Train'
    elif PID in val_set:
        split = 'Val'
    else:
        split = 'Test'        
    pd_data.append([PID, right_or_left, split, folder])


for _, row in sag_3d_dess_left_cases.iterrows():
    folder, PID, right_or_left = row['Folder'], row['ParticipantID'], 'RIGHT' if row['SeriesDescription'] == 'SAG_3D_DESS_RIGHT' else 'LEFT'
    split = '?'
    if PID in train_set:
        split = 'Train'
    elif PID in val_set:
        split = 'Val'
    else:
        split = 'Test'        
    pd_data.append([PID, right_or_left, split, folder])


# pd_data_frame = pd.DataFrame(sorted(pd_data))
# out_csv_file = '/playpen-raid2/qinliu/projects/iSegEngine/experiments/oai/oai_sag_3d_dess.csv'
# pd_data_frame.to_csv(out_csv_file,  header=['pid', 'right_or_left', 'split', 'folder'], index=False)

Split for ZIB:  407 50 50
Split for OAI:  3838 479 479


In [1]:
import sys
import SimpleITK as sitk
import numpy as np


def read_dicom_series(data_directory):
  """
  Read Dicom series from disk
  :param data_directory:  the folder containing all dicom series
  :return image: the 3D image volume
  """
  # Read the original series. First obtain the series file names using the image series reader.
  series_IDs = sitk.ImageSeriesReader.GetGDCMSeriesIDs(data_directory)
  if not series_IDs:
    print("ERROR: given directory \"" + data_directory + "\" does not contain a DICOM series.")
    sys.exit(1)
  series_file_names = sitk.ImageSeriesReader.GetGDCMSeriesFileNames(data_directory, series_IDs[0])

  series_reader = sitk.ImageSeriesReader()
  series_reader.SetFileNames(series_file_names)

  # Configure the reader to load all of the DICOM tags (public + private):
  # By default tags are not loaded (saves time). If tags are loaded, the private tags are not loaded.
  # We explicitly configure the reader to load tags, including the private ones.
  series_reader.MetaDataDictionaryArrayUpdateOn()
  series_reader.LoadPrivateTagsOn()
  image = series_reader.Execute()

  return image


In [ ]:
# This code is to generate training images for OAI dataset.
# Note that there is no annotaiton for this dataset.

import os
from PIL import Image
import matplotlib.pyplot as plt
import SimpleITK as sitk
import pandas as pd
import shutil

split_name = 'Train'
folder_name = 'train'

# read oai dataset 
oai_dataset_file = '/playpen-raid2/qinliu/projects/iSegEngine/experiments/oai/oai_sag_3d_dess.csv'
oai_df = pd.read_csv(oai_dataset_file)

# get all the test pids
oai_split_df = oai_df[oai_df['split'] == split_name]
oai_split_df = oai_split_df[oai_split_df['right_or_left'] == 'RIGHT']
oai_split_pids = list(set(oai_split_df['pid']))
oai_split_pids.sort()

print('Number of unique pids with RIGHT leg: ', len(oai_split_pids))

# from pid to folder
pid_mapper = {}
for pid in oai_split_pids:
    temp = oai_df[oai_df['pid'] == pid]
    temp = temp[temp['right_or_left'] == 'RIGHT']
    folder = str(temp['folder'].values[-1])
    pid_mapper[str(pid)] = folder

image_folder = '/playpen-raid2/qinliu/data/OAI/{}_debug/image'.format(folder_name)
mask_folder = '/playpen-raid2/qinliu/data/OAI/{}_debug/annotations'.format(folder_name)
if os.path.isdir(image_folder):
    shutil.rmtree(image_folder)

if os.path.isdir(mask_folder):
    shutil.rmtree(mask_folder)

os.makedirs(image_folder, exist_ok=True)
os.makedirs(mask_folder, exist_ok=True)

# generate images for the test set
for pid in pid_mapper:
    print(pid)
    image_dicom_folder = os.path.join('/playpen-raid/data/OAI', pid_mapper[pid])
    image = read_dicom_series(image_dicom_folder)
    image_npy = sitk.GetArrayFromImage(image)

    for slice_idx in [40, 80, 120]:
        image_slice = image_npy[slice_idx]
        print(image_slice.shape)

        img = Image.fromarray(image_slice)
        if img.mode != 'RGB':
            img = img.convert('RGB')
        img.save(os.path.join(image_folder, '{}_{}.png'.format(pid, slice_idx)))

In [ ]:
# This code is to generate images and annotation pictures for the OAI-ZIB dataset.
# Note that there are only 507 volumes in this dataset. All have annotations.

import matplotlib.pyplot as plt
import SimpleITK as sitk
import pandas as pd
import shutil

split_name = 'Train'
folder_name = 'train'

# read oai dataset 
oai_dataset_file = '/playpen-raid2/qinliu/projects/iSegEngine/experiments/oai/oai_sag_3d_dess.csv'
oai_df = pd.read_csv(oai_dataset_file)

# get all the test pids
oai_test_df = oai_df[oai_df['split'] == split_name]
oai_test_pids_set = set(oai_test_df['pid'])

# get PIDs of OAI-ZIB dataset
zib_dataset_file = '/playpen-raid2/qinliu/data/OAI-ZIB/doc/oai_mri_paths.txt'
ZIB_PID_to_Folder = {}
with open(zib_dataset_file) as f:
    lines = f.readlines()
    for line in lines:
        PID, folder = line.split(' ')[:2]
        folder = folder.rstrip("\n")
        PID = int(PID[:-1])
        ZIB_PID_to_Folder[PID] = folder

zib_test_pids = set.intersection(set(ZIB_PID_to_Folder.keys()), oai_test_pids_set)
zib_test_pids = list(zib_test_pids)
zib_test_pids.sort()

import os
from PIL import Image

image_folder = '/playpen-raid2/qinliu/data/OAI-ZIB/{}_debug/image'.format(folder_name)
mask_folder = '/playpen-raid2/qinliu/data/OAI-ZIB/{}_debug/annotations'.format(folder_name)
if os.path.isdir(image_folder):
    shutil.rmtree(image_folder)

if os.path.isdir(mask_folder):
    shutil.rmtree(mask_folder)

os.makedirs(image_folder, exist_ok=True)
os.makedirs(mask_folder, exist_ok=True)

# generate images for the test set
for pid in zib_test_pids:
    print(pid)
    image_dicom_folder = os.path.join('/playpen-raid/data/OAI', ZIB_PID_to_Folder[pid])
    image = read_dicom_series(image_dicom_folder)
    image_npy = sitk.GetArrayFromImage(image)

    mask_mhd_path = os.path.join(
        '/playpen-raid2/qinliu/data/OAI-ZIB/segmentation_masks', 
        '{}.segmentation_masks.mhd'.format(pid)
    )
    mask = sitk.ReadImage(mask_mhd_path)
    mask_npy = sitk.GetArrayFromImage(mask)

    print(image.GetDirection(), mask.GetDirection())
    print(image_npy.shape, mask_npy.shape)
    print(image_npy.dtype, mask_npy.dtype)

    mask_npy = np.transpose(mask_npy, (2,0,1))
    mask_npy = np.flip(mask_npy, axis=(0,1))

    mask_cartilage = np.zeros_like(mask_npy)
    mask_cartilage[mask_npy == 2] = 128
    mask_cartilage[mask_npy == 4] = 255

    for slice_idx in [40, 80, 120]:
        mask_slice = mask_cartilage[slice_idx]
        image_slice = image_npy[slice_idx]
        print(mask_slice.shape, image_slice.shape)

        if np.count_nonzero(mask_slice) > 20:
            img = Image.fromarray(image_slice)
            if img.mode != 'RGB':
                img = img.convert('RGB')
            img.save(os.path.join(image_folder, '{}_{}.png'.format(pid, slice_idx)))

            msk = Image.fromarray(mask_slice)
            if msk.mode != 'RGB':
                msk = msk.convert('RGB')
            msk.save(os.path.join(mask_folder, '{}_{}.png'.format(pid, slice_idx)))

    # print(image_npy.shape, mask_npy.shape)
    # plt.subplot(131)
    # plt.imshow(image_npy[50], cmap='gray')
    # plt.subplot(132)
    # plt.imshow(mask_npy[50], cmap='gray')
    # plt.show()

In [ ]:
# This code is to generate images and annotation volumes for the OAI-ZIB dataset.
import matplotlib.pyplot as plt
import SimpleITK as sitk
import pandas as pd
import shutil

split_name = 'Test'
folder_name = 'test'

# read oai dataset 
oai_dataset_file = '/playpen-raid2/qinliu/projects/iSegEngine/experiments/oai/oai_sag_3d_dess.csv'
oai_df = pd.read_csv(oai_dataset_file)

# get all the test pids
oai_test_df = oai_df[oai_df['split'] == split_name]
oai_test_pids_set = set(oai_test_df['pid'])

# get PIDs of OAI-ZIB dataset
zib_dataset_file = '/playpen-raid2/qinliu/data/OAI-ZIB/doc/oai_mri_paths.txt'
ZIB_PID_to_Folder = {}
with open(zib_dataset_file) as f:
    lines = f.readlines()
    for line in lines:
        PID, folder = line.split(' ')[:2]
        folder = folder.rstrip("\n")
        PID = int(PID[:-1])
        ZIB_PID_to_Folder[PID] = folder

zib_test_pids = set.intersection(set(ZIB_PID_to_Folder.keys()), oai_test_pids_set)
zib_test_pids = list(zib_test_pids)
zib_test_pids.sort()

import os
from PIL import Image

image_folder = '/playpen-raid2/qinliu/data/OAI-ZIB/{}_volumes_debug'.format(folder_name)
mask_folder = '/playpen-raid2/qinliu/data/OAI-ZIB/{}_volumes_debug'.format(folder_name)
if os.path.isdir(image_folder):
    shutil.rmtree(image_folder)

if os.path.isdir(mask_folder):
    shutil.rmtree(mask_folder)

os.makedirs(image_folder, exist_ok=True)
os.makedirs(mask_folder, exist_ok=True)

# generate images for the test set
for pid in zib_test_pids:
    print(pid)
    image_dicom_folder = os.path.join('/playpen-raid/data/OAI', ZIB_PID_to_Folder[pid])
    image = read_dicom_series(image_dicom_folder)
    image_npy = sitk.GetArrayFromImage(image)

    mask_mhd_path = os.path.join(
        '/playpen-raid2/qinliu/data/OAI-ZIB/segmentation_masks', 
        '{}.segmentation_masks.mhd'.format(pid)
    )
    mask = sitk.ReadImage(mask_mhd_path)
    mask_npy = sitk.GetArrayFromImage(mask)
    mask_npy = np.transpose(mask_npy, (2,0,1))
    mask_npy = np.flip(mask_npy, axis=(0,1))

    mask_aligned = sitk.GetImageFromArray(mask_npy)
    mask_aligned.CopyInformation(image)

    print(image_npy.dtype, mask_npy.dtype)

    # save image and mask
    mask_path = os.path.join(mask_folder, '{}_label.nii.gz'.format(pid))
    sitk.WriteImage(mask_aligned, mask_path)

    image_path = os.path.join(image_folder, '{}_image.nii.gz'.format(pid))
    sitk.WriteImage(image, image_path)
